In [1]:
# import os
# import gc
# import re
# import ast
# import sys
# import copy
# import json
# import time
# import math
# import string
# import pickle
# import random
# import joblib
# import itertools
# import warnings
# warnings.filterwarnings("ignore")

# import scipy as sp
# import numpy as np
# import pandas as pd
# from tqdm.auto import tqdm
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

# import torch
# import torch.nn as nn
# from torch.nn import Parameter
# import torch.nn.functional as F
# from torch.optim import Adam, SGD, AdamW
# from torch.utils.data import DataLoader, Dataset


# import tokenizers
# import transformers
# from transformers import AutoTokenizer, AutoModel, AutoConfig
# from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
# from transformers import DataCollatorWithPadding
# os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# import codecs
# from typing import Dict, List, Tuple
# from text_unidecode import unidecode

In [ ]:
# !pip install transformers==4.21.2
# !pip install tokenizers==0.12.1

# !pip install -q joblib scikit-learn scipy 

In [2]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

os.system('pip install iterative-stratification==0.1.7')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset


import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple


tokenizers.__version__: 0.12.1
transformers.__version__: 4.21.2
env: TOKENIZERS_PARALLELISM=true


# CFG


In [3]:
DATA_P = '/root/autodl-tmp/fb3/inputs/'
OUTPUT_DIR = '/root/autodl-tmp/fb3/output/trained_tiny_model/lsg-electra-base/'

In [4]:
# ====================================================
# CFG
# ====================================================
class CFG:
    wandb=False
    # competition='FB3'
    # _wandb_kernel='nakama'
    
    path=f"{DATA_P}common-nlp-tokenizer/model_tokenizer/tiny/lsg-electra-base/"
    model=path
    config_path=model+'config.pth'
    tokenizer = AutoTokenizer.from_pretrained(model)
    cfg_save_output = OUTPUT_DIR

    debug=False
    train=True
    trust_remote_code=True
    
    apex=True
    print_freq=20
    num_workers=4
    gradient_checkpointing=True
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=4
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=8
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    max_len=1436
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0,1]

In [5]:
# # ====================================================
# # tokenizer
# # ====================================================
# tokenizer = AutoTokenizer.from_pretrained(CFG.model)
# tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
# CFG.tokenizer = tokenizer

# Utils


In [6]:
# ====================================================
# Utils
# ====================================================

def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

##  Data Loading

In [7]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv(f'{DATA_P}feedback-prize-english-language-learning/train.csv')
test = pd.read_csv(f'{DATA_P}feedback-prize-english-language-learning/test.csv')
submission = pd.read_csv(f'{DATA_P}feedback-prize-english-language-learning/sample_submission.csv')


In [8]:
# ====================================================
# CV split
# ====================================================
Fold = MultilabelStratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(train, train[CFG.target_cols])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())

fold
0    978
1    977
2    978
3    978
dtype: int64

In [9]:
if CFG.debug:
    display(train.groupby('fold').size())
    train = train.sample(n=1000, random_state=0).reset_index(drop=True)
    display(train.groupby('fold').size())

# Dataset

In [10]:
# ====================================================
# Define max_len
# ====================================================
# lengths = []
# tk0 = tqdm(train['full_text'].fillna("").values, total=len(train))
# for text in tk0:
#     length = len(CFG.tokenizer(text, add_special_tokens=False)['input_ids'])
#     lengths.append(length)
# CFG.max_len = max(lengths) + 3 # cls & sep & sep
LOGGER.info(f"max_len: {CFG.max_len}")

  0%|          | 0/3911 [00:00<?, ?it/s]

max_len: 1436


In [11]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer.encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True, 
        max_length=CFG.max_len,
        pad_to_max_length=True,
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['full_text'].values
        self.labels = df[cfg.target_cols].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        label = torch.tensor(self.labels[item], dtype=torch.float)
        return inputs, label
    

def collate(inputs):
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
    return inputs

# Model

In [12]:
# ====================================================
# Model
# ====================================================
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings
    

class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True, trust_remote_code=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config, trust_remote_code=True)
        else:
            self.model = AutoModel(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()
        self.pool = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, 6)
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        feature = self.pool(last_hidden_states, inputs['attention_mask'])
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output
    
    
# ====================================================
# Loss
# ====================================================
class RMSELoss(nn.Module):
    def __init__(self, reduction='mean', eps=1e-9):
        super().__init__()
        self.mse = nn.MSELoss(reduction='none')
        self.reduction = reduction
        self.eps = eps

    def forward(self, y_pred, y_true):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        if self.reduction == 'none':
            loss = loss
        elif self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()
        return loss
    
    
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, labels) in enumerate(train_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs)
            loss = criterion(y_preds, labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
        if CFG.wandb:
            wandb.log({f"[fold{fold}] loss": losses.val,
                       f"[fold{fold}] lr": scheduler.get_lr()[0]})
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(inputs)
            loss = criterion(y_preds, labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

# train loop

In [13]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_cols].values
    
    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, OUTPUT_DIR+'config.pth')
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.SmoothL1Loss(reduction='mean') # RMSELoss(reduction="mean")
    
    best_score = np.inf

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
        
        # scoring
        score, scores = get_score(valid_labels, predictions)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}  Scores: {scores}')
        if CFG.wandb:
            wandb.log({f"[fold{fold}] epoch": epoch+1, 
                       f"[fold{fold}] avg_train_loss": avg_loss, 
                       f"[fold{fold}] avg_val_loss": avg_val_loss,
                       f"[fold{fold}] score": score})
        
        if best_score > score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        f"{CFG.cfg_save_output}_fold{fold}_best.pth")
                        # OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")

            cfg_save_output = f"{OUTPUT_DIR}trained_tiny_model/lsg-electra-base/"
            
            
    # predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
    #                          map_location=torch.device('cpu'))['predictions']
    predictions = torch.load(f"{CFG.cfg_save_output}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_folds[[f"pred_{c}" for c in CFG.target_cols]] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [14]:
CFG.model.split('/')[-2]

'lsg-electra-base'

In [15]:
if __name__ == '__main__':
    
    def get_result(oof_df):
        labels = oof_df[CFG.target_cols].values
        preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
        score, scores = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')
    
    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        oof_df.to_pickle(OUTPUT_DIR+'oof_df.pkl')
        
    print('finised')

========== fold: 0 training ==========
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
LSGElectraConfig {
  "_name_or_path": "/root/autodl-tmp/lsg-electra-base/",
  "adaptive": true,
  "architectures": [
    "LSGElectraForPreTraining"
  ],
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.0,
  "auto_map": {
    "AutoModel": "modeling_lsg_electra.LSGElectraModel",
    "AutoModelForCausalLM": "modeling_lsg_electra.LSGElectraForCausalLM",
    "AutoModelForMaskedLM": "modeling_lsg_electra.LSGElectraForMaskedLM",
    "AutoModelForMultipleChoice": "modeling_lsg_electra.LSGElectraForMultipleChoice",
    "AutoModelForPreTraining": "modeling_lsg_electra.LSGElectraForPreTraining",
    "AutoModelForQuestionAnswering": "modeling_lsg_electra.LSGElectraForQuestionAnswering",
    "AutoModelForSequenceClassification": "modeling_lsg_electra.LSGElectraForSequenceClassification"

Epoch: [1][0/366] Elapsed 0m 1s (remain 8m 37s) Loss: 2.4448(2.4448) Grad: inf  LR: 0.00002000  
Epoch: [1][20/366] Elapsed 0m 7s (remain 2m 5s) Loss: 1.9463(2.1084) Grad: 213526.2656  LR: 0.00001999  
Epoch: [1][40/366] Elapsed 0m 13s (remain 1m 45s) Loss: 0.3212(1.4827) Grad: 127912.7344  LR: 0.00001996  
Epoch: [1][60/366] Elapsed 0m 19s (remain 1m 38s) Loss: 0.2921(1.0789) Grad: 112994.8750  LR: 0.00001991  
Epoch: [1][80/366] Elapsed 0m 26s (remain 1m 32s) Loss: 0.2104(0.8637) Grad: 96628.5000  LR: 0.00001985  
Epoch: [1][100/366] Elapsed 0m 32s (remain 1m 25s) Loss: 0.1454(0.7255) Grad: 125618.9844  LR: 0.00001977  
Epoch: [1][120/366] Elapsed 0m 38s (remain 1m 18s) Loss: 0.2384(0.6324) Grad: 58255.9219  LR: 0.00001967  
Epoch: [1][140/366] Elapsed 0m 45s (remain 1m 12s) Loss: 0.1064(0.5626) Grad: 49054.6484  LR: 0.00001955  
Epoch: [1][160/366] Elapsed 0m 51s (remain 1m 5s) Loss: 0.1971(0.5096) Grad: 88765.9297  LR: 0.00001941  
Epoch: [1][180/366] Elapsed 0m 57s (remain 0m 58s)

Epoch 1 - avg_train_loss: 0.2973  avg_val_loss: 0.1226  time: 130s
Epoch 1 - Score: 0.4974  Scores: [0.5166656761352973, 0.4777127241669594, 0.47244990940846804, 0.4984803535383854, 0.5319704195334692, 0.4872868815061927]
Epoch 1 - Save Best Score: 0.4974 Model


EVAL: [60/62] Elapsed 0m 15s (remain 0m 0s) Loss: 0.1134(0.1227) 
EVAL: [61/62] Elapsed 0m 15s (remain 0m 0s) Loss: 0.0962(0.1226) 
Epoch: [2][0/366] Elapsed 0m 0s (remain 4m 24s) Loss: 0.1255(0.1255) Grad: 133688.5469  LR: 0.00001706  
Epoch: [2][20/366] Elapsed 0m 7s (remain 1m 59s) Loss: 0.1093(0.1068) Grad: 93737.2578  LR: 0.00001675  
Epoch: [2][40/366] Elapsed 0m 13s (remain 1m 48s) Loss: 0.1720(0.1177) Grad: 97595.2812  LR: 0.00001643  
Epoch: [2][60/366] Elapsed 0m 19s (remain 1m 37s) Loss: 0.0870(0.1197) Grad: 106552.6797  LR: 0.00001610  
Epoch: [2][80/366] Elapsed 0m 25s (remain 1m 29s) Loss: 0.1268(0.1220) Grad: 177491.2344  LR: 0.00001575  
Epoch: [2][100/366] Elapsed 0m 31s (remain 1m 23s) Loss: 0.1473(0.1209) Grad: 208036.2188  LR: 0.00001540  
Epoch: [2][120/366] Elapsed 0m 38s (remain 1m 17s) Loss: 0.0775(0.1198) Grad: 38357.0156  LR: 0.00001503  
Epoch: [2][140/366] Elapsed 0m 44s (remain 1m 11s) Loss: 0.1905(0.1193) Grad: 133978.7031  LR: 0.00001466  
Epoch: [2][160/

Epoch 2 - avg_train_loss: 0.1146  avg_val_loss: 0.1122  time: 131s
Epoch 2 - Score: 0.4749  Scores: [0.511003344931223, 0.46730272313591015, 0.42892431044116597, 0.47139263771343026, 0.5040661888428912, 0.46701069084870184]
Epoch 2 - Save Best Score: 0.4749 Model


...

EVAL: [60/62] Elapsed 0m 14s (remain 0m 0s) Loss: 0.0937(0.1086) 
EVAL: [61/62] Elapsed 0m 14s (remain 0m 0s) Loss: 0.0442(0.1084) 
Epoch: [4][0/366] Elapsed 0m 0s (remain 5m 32s) Loss: 0.0714(0.0714) Grad: 80821.5156  LR: 0.00000294  
Epoch: [4][20/366] Elapsed 0m 7s (remain 2m 9s) Loss: 0.1418(0.1026) Grad: 142979.8750  LR: 0.00000264  
Epoch: [4][40/366] Elapsed 0m 14s (remain 1m 51s) Loss: 0.2050(0.1047) Grad: 170244.7656  LR: 0.00000236  
Epoch: [4][60/366] Elapsed 0m 20s (remain 1m 40s) Loss: 0.0750(0.1015) Grad: 99660.9297  LR: 0.00000209  
Epoch: [4][80/366] Elapsed 0m 26s (remain 1m 33s) Loss: 0.1386(0.1010) Grad: 110946.5000  LR: 0.00000183  
Epoch: [4][100/366] Elapsed 0m 33s (remain 1m 27s) Loss: 0.0600(0.1001) Grad: 69135.7422  LR: 0.00000159  
Epoch: [4][120/366] Elapsed 0m 39s (remain 1m 19s) Loss: 0.0811(0.1007) Grad: 71444.8047  LR: 0.00000137  
Epoch: [4][140/366] Elapsed 0m 46s (remain 1m 13s) Loss: 0.0554(0.0991) Grad: 88400.7188  LR: 0.00000116  
Epoch: [4][160/366

Epoch 4 - avg_train_loss: 0.0962  avg_val_loss: 0.1082  time: 133s
Epoch 4 - Score: 0.4663  Scores: [0.5027185538033578, 0.4533606108598723, 0.4309384753262597, 0.4550886891511836, 0.4928231639087349, 0.4629594929363798]
Epoch 4 - Save Best Score: 0.4663 Model


EVAL: [60/62] Elapsed 0m 14s (remain 0m 0s) Loss: 0.0912(0.1083) 
EVAL: [61/62] Elapsed 0m 14s (remain 0m 0s) Loss: 0.0479(0.1082) 


========== fold: 3 result ==========
Score: 0.4663  Scores: [0.5027185538033578, 0.4533606108598723, 0.4309384753262597, 0.4550886891511836, 0.4928231639087349, 0.4629594929363798]
========== CV ==========
Score: 0.4724  Scores: [0.5061653231580762, 0.4624927400134135, 0.43269011127595514, 0.46738706941242064, 0.49494898435707757, 0.4708564904202183]


finised
